In [1]:
import math
import json
import requests
import itertools
import numpy as np

import pandas as pd
import time
import datetime as dt

In [2]:
# Pushshift API Github: https://github.com/pushshift/api
# Code mainly used from: https://medium.com/@pasdan/how-to-scrap-reddit-using-pushshift-io-via-python-a3ebcc9b83f4

# We need a method that takes in a URI and handles the HTTP request/response.
def make_request(uri, max_retries = 2):
    def fire_away(uri):
        response = requests.get(uri)
        assert response.status_code == 200
        return json.loads(response.content)

    current_tries = 1
    while current_tries < max_retries:
        try:
            time.sleep(1)
            response = fire_away(uri)
            return response
        except:
            time.sleep(1)
            current_tries += 1

    return fire_away(uri)

def map_posts(posts):
        return list(map(lambda post: {
            'id': post['id'],
            'time': post['created_utc'],
            'score': post['score'],
            'comments': post['num_comments']}, posts))

In [ ]:
size = 500
score = 200000
posts = 10000
subreddit = askreddit
iters = int(posts / size)

URI_TEMPLATE = 'https://api.pushshift.io/reddit/search/submission?subreddit={}&size={}&score=<{}&sort=acs&sort_type=score'
posts = map_posts(make_request(URI_TEMPLATE.format(subreddit, size, score))['data'])

for i in range(iters - 1):
    last = posts[-1]
    new_score = last['score']

    posts.extend(map_posts(make_request(URI_TEMPLATE.format(subreddit, size, new_score))['data']))

print(len(posts))

In [ ]:
posts_df = pd.DataFrame(posts)

In [4]:
def get_date(created):
    return dt.datetime.fromtimestamp(created)

_timestamp = posts_df["time"].apply(get_date)

posts_df = posts_df.assign(timestamp = _timestamp)

NameError: name 'posts_df' is not defined

In [ ]:
posts_df.to_csv('pushshift.csv', index=False) 

In [3]:
size = 500
posts = 1000000
#Random Date: Jul. 19, 2018
before = 1531972800
subreddit = 'askreddit'
iters = int(posts / size)

recent_10000 = 'https://api.pushshift.io/reddit/search/submission?subreddit={}&size={}&before={}&sort_type=created_utc'
recent_posts = map_posts(make_request(recent_10000.format(subreddit, size, before))['data'])

for i in range(iters - 1):
    last = recent_posts[-1]
    new_time = last['time']
    recent_posts.extend(map_posts(make_request(recent_10000.format(subreddit, size, new_time))['data']))



NameError: name 'get_date' is not defined

In [6]:
# Jul. 19, 2018
get_date(1270637661)

datetime.datetime(2010, 4, 7, 6, 54, 21)

In [5]:
recent_posts_df = pd.DataFrame(recent_posts)
_timestamp = recent_posts_df["time"].apply(get_date)

recent_posts_df = recent_posts_df.assign(timestamp = _timestamp)
recent_posts_df.to_csv('pushshift_recent.csv', index=False) 